# 3.3 线性回归的简洁实现

In [38]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.10.0


## 3.3.1 生成数据集

In [39]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
print(features)
print("---------------")
print(1*features[:,0])
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
print(labels)
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

tensor([[ 0.4115,  0.2415],
        [-0.3223, -1.9621],
        [ 0.1940,  1.0024],
        ...,
        [-0.9130,  1.2136],
        [ 0.3836,  2.3992],
        [ 0.0223,  0.3907]])
---------------
tensor([ 4.1152e-01, -3.2234e-01,  1.9399e-01,  1.0161e+00,  5.9281e-02,
        -2.1213e-01, -3.7864e-01, -4.8259e-01,  5.3169e-01, -2.1398e-01,
        -7.8904e-03,  8.6032e-01,  9.3127e-01, -3.9983e-01,  4.7720e-01,
         2.9700e-01, -9.7393e-01,  1.1238e+00, -1.8618e+00, -2.4606e+00,
         4.3461e-01,  8.9581e-01, -2.1615e-01, -1.1397e+00, -5.7995e-02,
        -2.1096e+00,  1.0056e+00,  1.5860e+00,  1.4062e+00,  3.4780e-01,
        -2.2013e+00, -8.4607e-01,  7.0519e-01,  1.5777e+00, -1.1110e+00,
        -1.2706e+00,  8.8376e-01,  1.0943e+00, -6.0772e-01,  1.0291e+00,
        -1.1207e+00, -8.4855e-01,  1.6276e+00,  9.0230e-01, -1.7352e-01,
         1.1453e+00, -8.0203e-01, -2.5025e+00,  5.4487e-01,  2.5880e-01,
         8.3449e-01, -1.4430e+00, -1.0537e+00,  8.8243e-01, -4.4723e-01,

## 3.3.2 读取数据

In [40]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

In [41]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[ 0.3813,  1.2292],
        [ 0.8603,  1.1902],
        [ 2.0161,  0.4783],
        [-0.6149,  0.3711],
        [ 0.7453, -0.6069],
        [ 0.1141, -0.6846],
        [-0.2762,  0.9166],
        [-0.8699, -0.9187],
        [ 0.5286, -1.6239],
        [-1.0629,  0.0205]]) 
 tensor([ 0.7877,  1.8675,  6.6013,  1.7099,  7.7469,  6.7555,  0.5196,  5.5721,
        10.7673,  1.9942])


## 3.3.3 定义模型

In [42]:
class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [43]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [44]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.2756, -0.0516]], requires_grad=True)
Parameter containing:
tensor([-0.0637], requires_grad=True)


## 3.3.4 初始化模型参数

In [45]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [46]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[-0.0103, -0.0056]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [47]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [48]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [49]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [50]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [51]:
num_epochs = 100
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000261
epoch 2, loss: 0.000122
epoch 3, loss: 0.000083
epoch 4, loss: 0.000033
epoch 5, loss: 0.000081
epoch 6, loss: 0.000232
epoch 7, loss: 0.000085
epoch 8, loss: 0.000029
epoch 9, loss: 0.000021
epoch 10, loss: 0.000079
epoch 11, loss: 0.000099
epoch 12, loss: 0.000035
epoch 13, loss: 0.000073
epoch 14, loss: 0.000072
epoch 15, loss: 0.000075
epoch 16, loss: 0.000100
epoch 17, loss: 0.000105
epoch 18, loss: 0.000207
epoch 19, loss: 0.000059
epoch 20, loss: 0.000108
epoch 21, loss: 0.000115
epoch 22, loss: 0.000092
epoch 23, loss: 0.000124
epoch 24, loss: 0.000170
epoch 25, loss: 0.000117
epoch 26, loss: 0.000062
epoch 27, loss: 0.000045
epoch 28, loss: 0.000105
epoch 29, loss: 0.000059
epoch 30, loss: 0.000082
epoch 31, loss: 0.000076
epoch 32, loss: 0.000064
epoch 33, loss: 0.000068
epoch 34, loss: 0.000092
epoch 35, loss: 0.000103
epoch 36, loss: 0.000126
epoch 37, loss: 0.000100
epoch 38, loss: 0.000115
epoch 39, loss: 0.000093
epoch 40, loss: 0.000083
epoch 41,

In [52]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 2.0002, -3.4006]])
4.2 tensor([4.1993])
